Suppose we want to check if the following two queries are equivalent:

query1: SELECT A.name FROM A LEFT JOIN B ON A.id = B.id;

query2: SELECT A.name FROM A JOIN B ON A.id = B.id;

Since these two queries are not equivalent, we're supposed to see sat, followed by a counterexample 

Step 1: declare variables for each query

In [48]:
from z3 import *

# declare variables for q1
A_q1_id = Int('A_q1_id')
A_q1_name = String('A_q1_name')
B_q1_id = Int('B_q1_id')
B_q1_name = String('B_q1_name')
# declare variables for q2
A_q2_id = Int('A_q2_id')
A_q2_name = String('A_q2_name')
B_q2_id = Int('B_q2_id')
B_q2_name = String('B_q2_name')
# define what NULL is 
NULL = IntVal(-1) 
# actually no matter which value I choose, there are always other values that are not equal to it
# which helps distinguish OUTER JOIN from INNER JOIN

Step 2: initialize the solver, and add constraints on that the inputs for q1 and q2 are equal. 

In [49]:
s = Solver()
s.add(A_q1_id == A_q2_id)
s.add(A_q1_name == A_q2_name)
s.add(B_q1_id == B_q2_id)
s.add(B_q1_name == B_q2_name)

Step 3: encode q1 and q2 respectively.

During encoding q2, we create a Z3.function LeftJoin_int, which takes two ints and returns a boolean.

Then, we add the following two constraints:

1.when the condition holds (e.g. A.id = B.id), then LeftJoin_int(A_q2_id, B_q2_id) = true.

2.when the condition doesn't hold (e.g. there's no tuple in B such that A.id = B.id), then LeftJoin_int(A_q2_id, NULL) = true.

In [50]:
# encode q1
cond_q1 = And(True, A_q1_id == B_q1_id) # the first True comes from "no explicit join"

# encode q2
LeftJoin_int = Function('LeftJoin_int', IntSort(), IntSort(), BoolSort())
# 1) matching pairs
cond_q2 = Implies(A_q2_id == B_q2_id, LeftJoin_int(A_q2_id, B_q2_id))
# 2) unmatched A: output (A, NULL)
cond_q2 = And(cond_q2, Implies(A_q2_id != B_q2_id, LeftJoin_int(A_q2_id, NULL)))
# during real encoding process, will need to choose function based on the type 
# e.g. if we perform left join on A.name = B.name, then use LeftJoin_string

Step 4: combine encodings for both query and add them to the solver

In [51]:
q1_result = Bool("q1_result")
q2_result = Bool("q2_result")
s.add(q1_result == cond_q1)
s.add(q2_result == cond_q2)
s.add(q1_result != q2_result)

Step 5: solve and get the model

In [52]:
print(s.check())
print(s.model())

sat
[A_q2_id = 0,
 B_q2_id = 1,
 q1_result = False,
 B_q2_name = "",
 B_q1_name = "",
 A_q2_name = "",
 A_q1_name = "",
 A_q1_id = 0,
 B_q1_id = 1,
 q2_result = True,
 LeftJoin_int = [(0, -1) -> True, else -> False]]


That was a simple example of how to encode left join (i.e. only one condition in the ON clause), but how about more complicated ones?

Now let's assume we have two new queries:

query3: SELECT A.name FROM A LEFT JOIN B ON A.id = B.id WHERE B.price >= 3;

query4: SELECT A.name FROM A LEFT JOIN B ON A.id = B.id AND B.price >= 3;

Note that the difference in these two queries is that I replaced 'WHERE' with 'AND'. This would not cause a behavioral change for INNER JOIN, but it does for OUTER JOIN. 

Again, we expect to see sat and a counterexample.

One counterexample could be:

A: {123, "product X"}, {234, "product Y"}

B: {234, 1} {234, 10}

query3 output: {"product Y"}

query4 output: {"product X"}, {"product Y"}

Step 1: declare variables for each query

In [53]:
# synthetic row identifiers; completely invented
A_row = Int('A_row')
B_row = Int('B_row')

# declare variables for q3
A_q3_id = Int('A_q3_id')
A_q3_name = String('A_q3_name')
B_q3_id = Int('B_q3_id')
B_q3_price = Real('B_q3_price')
# declare variables for q4
A_q4_id = Int('A_q4_id')
A_q4_name = String('A_q4_name')
B_q4_id = Int('B_q4_id')
B_q4_price = Real('B_q4_price')

Step 2: initialize the solver, and add constraints on that the inputs for q1 and q2 are equal. 

In [54]:
s = Solver()
s.add(A_q3_id == A_q4_id)
s.add(A_q3_name == A_q4_name)
s.add(B_q3_id == B_q4_id)
s.add(B_q3_price == B_q4_price)

Step 3: encode q1 and q2 respectively.

The encoding for q3 looks very similar to that of q4, while we see something different when encoding q4.

In [ ]:
# encode q3: SELECT A.name FROM A LEFT JOIN B ON A.id = B.id WHERE B.price >= 3;
LeftJoin = Function('LeftJoin', IntSort(), IntSort(), BoolSort())
# cond_join_q3 = Or(
#     And(A_q3_id == B_q3_id, LeftJoin(A_row, B_row)),
#     And(Not(A_q3_id == B_q3_id), LeftJoin(A_row, NULL))
# )
cond_q3 = And(
    A_q3_id == B_q3_id,   # must be a match
    B_q3_price >= 3       # must satisfy WHERE
)


# encode q4: SELECT A.name FROM A LEFT JOIN B ON A.id = B.id AND B.price >= 3;
join_cond_q4 = And(A_q4_id == B_q4_id, B_q4_price >= 3)
LeftJoin = Function('LeftJoin', IntSort(), IntSort(), BoolSort())
cond_q4 = Implies(
    join_cond_q4,
    LeftJoin(A_row, B_row)
)
# unmatched → include NULL
cond_q4 = And(cond_q4,
    Implies(
        Not(join_cond_q4),
        LeftJoin(A_row, NULL)
    )
)
# finish encoding JOIN clause, no additional constraint from WHERE clause
# end


# how to encode q5 (SELECT A.name FROM A LEFT JOIN B ON A.id = B.id OR B.price >= 3;)?
# join_cond_5 = Or(A_q5_id == B_q5_id, B_q5_price >= 3)
# cond_q5 = Implies(cond_q5, LeftJoin(A_row, B_row))
# cond_q5 = And(cond_q4, Implies(not join_cond_4, LeftJoin(A_row, NULL)))

print(f"cond_q3: {cond_q3}")
print(f"cond_q4: {cond_q4}")

cond_q3: And(A_q3_id == B_q3_id, B_q3_price >= 3)
cond_q4: And(Implies(And(A_q4_id == B_q4_id, B_q4_price >= 3),
            LeftJoin(A_row, B_row)),
    Implies(Not(And(A_q4_id == B_q4_id, B_q4_price >= 3)),
            LeftJoin(A_row, -1)))


Step 4: combine encodings for both query and add them to the solver

Step 5: solve and get the model

In [57]:
q3_result = Bool("q3_result")
q4_result = Bool("q4_result")
s.add(q3_result == cond_q3)
s.add(q4_result == cond_q4)
s.add(q3_result != q4_result)

print(s.check())
print(s.model())

sat
[A_q4_name = "!0!",
 q4_result = True,
 A_q3_name = "!0!",
 B_q4_price = 0,
 A_q3_id = 2,
 A_q4_id = 2,
 A_row = 4,
 B_q4_id = 3,
 B_q3_id = 3,
 q3_result = False,
 B_q3_price = 0,
 LeftJoin = [else -> True]]


<!-- 
sat
[A_row = 0,
 A_q4_id = 1,
 B_row = 1,
 B_q4_price = 3,
 B_q4_id = 0,
 A_q3_id = 1,
 q3_result = False,
 B_q3_id = 0,
 B_q3_price = 3,
 A_q4_name = "",
 A_q3_name = "",
 q4_result = True,
 LeftJoin = [(0, -1) -> True, else -> False]]
 -->

Analysis: when 

A: {0, "!0!"}

B: {0, 3}

query4 outputs "" while query3 doesn't